# Overwatch Competitive Analytics

**Prima parte**: connessione a MongoDB

In [1]:
from pymongo import MongoClient

# Connessione al database MongoDB
client = MongoClient("mongodb+srv://<usr>:<pw>@<host>")
db = client["Overwatch2"]
season1 = db["SEASON1"]
season2 = db["SEASON2"]
season3 = db["SEASON3"]
season4 = db["SEASON4"]


**Seconda parte**: creiamo una collection temporanea con tutti i dati

In [2]:
# Unire le collezioni delle quattro stagioni
all_seasons = db["ALL_SEASONS"]
all_seasons.delete_many({})  # Pulire la collezione temporanea se esiste già

# Creare una lista per i documenti di tutte le stagioni
documents = []

# Funzione per aggiungere stagione al documento
def add_season_info(doc, season):
    doc["Season"] = season
    return doc

for collection, season_name in zip([season1, season2, season3, season4], ["SEASON1", "SEASON2", "SEASON3", "SEASON4"]):
    documents.extend([add_season_info(doc, season_name) for doc in collection.find()])

# Inserire tutti i documenti in una sola operazione
all_seasons.insert_many(documents)


InsertManyResult([ObjectId('66a0fb4033e7e17b516b12e0'), ObjectId('66a0fb4033e7e17b516b12e1'), ObjectId('66a0fb4033e7e17b516b12e2'), ObjectId('66a0fb4033e7e17b516b12e3'), ObjectId('66a0fb4033e7e17b516b12e4'), ObjectId('66a0fb4033e7e17b516b12e5'), ObjectId('66a0fb4033e7e17b516b12e6'), ObjectId('66a0fb4033e7e17b516b12e7'), ObjectId('66a0fb4033e7e17b516b12e8'), ObjectId('66a0fb4033e7e17b516b12e9'), ObjectId('66a0fb4033e7e17b516b12ea'), ObjectId('66a0fb4033e7e17b516b12eb'), ObjectId('66a0fb4033e7e17b516b12ec'), ObjectId('66a0fb4033e7e17b516b12ed'), ObjectId('66a0fb4033e7e17b516b12ee'), ObjectId('66a0fb4033e7e17b516b12ef'), ObjectId('66a0fb4033e7e17b516b12f0'), ObjectId('66a0fb4033e7e17b516b12f1'), ObjectId('66a0fb4033e7e17b516b12f2'), ObjectId('66a0fb4033e7e17b516b12f3'), ObjectId('66a0fb4033e7e17b516b12f4'), ObjectId('66a0fb4033e7e17b516b12f5'), ObjectId('66a0fb4033e7e17b516b12f6'), ObjectId('66a0fb4033e7e17b516b12f7'), ObjectId('66a0fb4033e7e17b516b12f8'), ObjectId('66a0fb4033e7e17b516b12

**Terza parte**: analisi con aggregazioni  
Utilizzare l'aggregation framework per calcolare le medie di `KDA Ratio`, `Win Rate, %`, `Damage / 10min`, `Healing / 10min`, `Deaths / 10min` per ciascun eroe.

In [21]:
pipelineTotal = [
    {"$group": {
        "_id": "$Hero",
        "avg_KDA_Ratio": {"$avg": "$KDA Ratio"},
        "avg_Win_Rate": {"$avg": "$Win Rate, %"},
        "avg_Damage_per_10min": {"$avg": "$Damage / 10min"},
        "avg_Healing_per_10min": {"$avg": "$Healing / 10min"},
        "avg_Deaths_per_10min": {"$avg": "$Deaths / 10min"}
    }},
    {"$sort": {"avg_Win_Rate": -1}}
]

result = all_seasons.aggregate(pipelineTotal)
for doc in result:
    print(doc)


{'_id': 'Torbjorn', 'avg_KDA_Ratio': 3.1193750000000002, 'avg_Win_Rate': 53.4746875, 'avg_Damage_per_10min': 9491.0625, 'avg_Healing_per_10min': None, 'avg_Deaths_per_10min': 6.9643749999999995}
{'_id': 'Symmetra', 'avg_KDA_Ratio': 3.575, 'avg_Win_Rate': 52.545625, 'avg_Damage_per_10min': 8287.34375, 'avg_Healing_per_10min': None, 'avg_Deaths_per_10min': 8.020625}
{'_id': 'Ramattra', 'avg_KDA_Ratio': 3.7570833333333336, 'avg_Win_Rate': 51.03, 'avg_Damage_per_10min': 9559.291666666666, 'avg_Healing_per_10min': None, 'avg_Deaths_per_10min': 6.655833333333334}
{'_id': 'Reinhardt', 'avg_KDA_Ratio': 3.02, 'avg_Win_Rate': 50.808125, 'avg_Damage_per_10min': 8002.125, 'avg_Healing_per_10min': None, 'avg_Deaths_per_10min': 7.125625}
{'_id': 'Doomfist', 'avg_KDA_Ratio': 3.2421875, 'avg_Win_Rate': 50.2528125, 'avg_Damage_per_10min': 7659.1875, 'avg_Healing_per_10min': None, 'avg_Deaths_per_10min': 7.80875}
{'_id': 'Zenyatta', 'avg_KDA_Ratio': 5.12, 'avg_Win_Rate': 49.9228125, 'avg_Damage_per_10mi

Trovare l'eroe con il tasso di vittoria più alto e quello con il più basso

In [6]:
pipeline = [
    {"$group": {
        "_id": "$Hero",
        "avg_Win_Rate": {"$avg": "$Win Rate, %"}
    }},
    {"$sort": {"avg_Win_Rate": -1}},
    {"$limit": 1}
]

highest_win_rate = list(all_seasons.aggregate(pipeline))[0]
print(f"Eroe con il tasso di vittoria più alto: {highest_win_rate}")

pipeline[1]["$sort"]["avg_Win_Rate"] = 1
lowest_win_rate = list(all_seasons.aggregate(pipeline))[0]
print(f"Eroe con il tasso di vittoria più basso: {lowest_win_rate}")


Eroe con il tasso di vittoria più alto: {'_id': 'Torbjorn', 'avg_Win_Rate': 53.4746875}
Eroe con il tasso di vittoria più basso: {'_id': 'Lifeweaver', 'avg_Win_Rate': 38.455}


Creare un indice sui campi `Hero` e `Role`

In [7]:
all_seasons.create_index([("Hero", 1)])
all_seasons.create_index([("Role", 1)])

'Role_1'

Eseguire una query per trovare tutti i dati degli eroi di ruolo Support

In [8]:
support_heroes = all_seasons.find({"Role": "Support"}).explain()
print(support_heroes)

{'explainVersion': '1', 'queryPlanner': {'namespace': 'Overwatch2.ALL_SEASONS', 'indexFilterSet': False, 'parsedQuery': {'Role': {'$eq': 'Support'}}, 'queryHash': '8F0D14DF', 'planCacheKey': '8AADE24F', 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'winningPlan': {'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'Role': 1}, 'indexName': 'Role_1', 'isMultiKey': False, 'multiKeyPaths': {'Role': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'Role': ['["Support", "Support"]']}}}, 'rejectedPlans': []}, 'executionStats': {'executionSuccess': True, 'nReturned': 264, 'executionTimeMillis': 0, 'totalKeysExamined': 264, 'totalDocsExamined': 264, 'executionStages': {'stage': 'FETCH', 'nReturned': 264, 'executionTimeMillisEstimate': 1, 'works': 265, 'advanced': 264, 'needTime': 0, 'needYield': 0, 'saveState': 0, 'restoreState': 0, 'isEOF'

Creare un dataframe Pandas con questi dati

In [22]:
import pandas as pd

data = list(all_seasons.aggregate(pipelineTotal))
df = pd.DataFrame(data)
df

,_id,avg_KDA_Ratio,avg_Win_Rate,avg_Damage_per_10min,avg_Healing_per_10min,avg_Deaths_per_10min
0,Torbjorn,3.119375,53.474688,9491.062500,NaN,6.964375
1,Symmetra,3.575000,52.545625,8287.343750,NaN,8.020625
2,Ramattra,3.757083,51.030000,9559.291667,NaN,6.655833
3,Reinhardt,3.020000,50.808125,8002.125000,NaN,7.125625
4,Doomfist,3.242188,50.252812,7659.187500,NaN,7.808750
5,Zenyatta,5.120000,49.922812,4802.156250,6199.12500,7.251250
6,Sigma,4.771875,49.873125,9701.000000,NaN,5.315313
7,Genji,2.494375,49.638438,7316.718750,NaN,8.248437
8,Mei,3.895313,49.636250,6918.593750,NaN,7.340000
9,Lucio,3.797188,49.445000,3525.000000,7531.75000,7.504375
